In [1]:
from pitchr import data
from pitchr import xml_parser
from pitchr import harmony_maker
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, Dropout
DATA_PATH = "../dataset/_xml_scores"
score_files = os.listdir(DATA_PATH)

In [29]:
path = "../dataset/_xml_scores"
score_files = os.listdir(path)

while ".DS_Store" in score_files:
    score_files.remove(".DS_Store")

score_name = "All_I_Want_For_Christmas_Is_You"
file_name = "score.xml"
target = (f"{path}/{score_name}/{file_name}")
infile = open(target, 'r', encoding='utf-8')
contents = infile.read()
infile.close()
melody_dfs, harmony_dfs = xml_parser.parse_xml(contents)
print("length of melody_dfs and harmony_dfs:", len(melody_dfs), ", ", len(harmony_dfs))
print(type(melody_dfs))

length of melody_dfs and harmony_dfs: 11 ,  11
<class 'list'>


In [86]:
all_melody_dfs, all_harmony_dfs = xml_parser.get_all_data()
print(len(all_melody_dfs), len(all_harmony_dfs))
for i in range(0, 210):
    print(len(all_melody_dfs[i]), len(all_harmony_dfs[i]))

210 210
50 8
50 10
50 22
50 42
50 43
50 32
50 42
50 43
50 36
50 42
50 43
50 58
50 52
50 67
50 57
50 47
50 50
50 38
50 95
50 42
50 73
50 25
50 52
50 42
50 51
50 53
50 59
50 48
50 47
50 40
50 16
50 19
50 24
50 39
50 42
50 40
50 31
50 58
50 70
50 51
50 44
50 77
50 60
50 66
50 36
50 57
50 55
50 54
50 54
50 69
50 54
50 60
50 48
50 51
50 51
50 40
50 54
50 44
50 47
50 46
50 49
50 54
50 288
50 70
50 12
50 57
50 53
50 46
50 66
50 35
50 80
50 38
50 169
50 27
50 73
50 29
50 159
50 80
50 67
50 53
50 33
50 34
50 50
50 48
50 69
50 66
50 46
50 111
50 88
50 7
50 28
50 73
50 73
50 82
50 61
50 29
50 61
50 17
50 44
50 28
50 44
50 17
50 18
50 44
50 19
50 31
50 16
50 18
50 40
50 18
50 47
50 52
50 68
50 65
50 60
50 64
50 48
50 28
50 31
50 27
50 22
50 16
50 16
50 39
50 22
50 29
50 16
50 21
50 27
50 33
50 31
50 31
50 22
50 21
50 51
50 7
50 6
50 4
50 4
50 99
50 35
50 70
50 37
50 29
50 91
50 50
50 28
50 32
50 28
50 28
50 41
50 32
50 24
50 20
50 28
50 34
50 40
50 74
50 59
50 86
50 54
50 58
50 61
50 54
50 88
50 2

In [81]:
# Converting dfs to numpy arrays
minimum_melody = 999
minimum_harmony = 999
all_melody_np = []
all_harmony_np = []
for df in all_melody_dfs:
    df = df[['Pitch Number', 'Pitch Interval', 'Pitch Predictability']]
    temp_np = df.to_numpy()
    all_melody_np.append(temp_np)
for df in all_harmony_dfs:
    df = df[['Pitch Number', 'Pitch Interval', 'Pitch Predictability']]
    temp_np = df.to_numpy()
    all_harmony_np.append(temp_np)


print(type(all_melody_np))
print(type(all_harmony_np))

<class 'list'>
<class 'list'>


In [85]:
# convert dataframes to numpy arrays
all_melody_np = np.asarray(all_melody_np)
print(type(all_melody_np))
all_harmony_np = np.array(all_harmony_np).reshape(210, -1, 3)
print(type(all_melody_np))
print(type(all_harmony_np))
print(all_melody_np.shape)
print(all_harmony_np.shape)
# harmony notes are giving me problems. Might have to pad them

<class 'numpy.ndarray'>


ValueError: cannot reshape array of size 210 into shape (210,newaxis,3)

In [73]:
# convert all values to floats
# min pitch number is -33
# max pitch number is 43
for array in all_melody_np:
    for note in array:
        for i in range(0, 3):
            if type(note[i]) == str:
                note[i] = float(-50)
                np.float32
            elif type(note[i]) == int:
                note[i] = float(note[i])
for array in all_harmony_np:
    for note in array:
        for i in range(0, 3):
            if type(note[i]) == str:
                note[i] = float(-50)
            elif type(note[i]) == int:
                note[i] = float(note[i])


print(all_melody_np.shape)
print(all_harmony_np.shape)

(210, 50, 3)
(210,)


In [73]:
melody_train, melody_validation, melody_test, harmony_train, harmony_validation, harmony_test = harmony_maker.prepare_data(.2, .1)

AttributeError: 'list' object has no attribute 'iloc'

In [5]:
print(type(melody_train))
print(melody_train.shape)
print(melody_train[0].shape)
print(type(harmony_train))
print(harmony_train.shape)
print(harmony_train[1].shape)

<class 'numpy.ndarray'>
(29,)
(237, 3)
<class 'numpy.ndarray'>
(29,)
(143, 3)


In [34]:
model = harmony_maker.build_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (210, 50, 512)            1056768   
_________________________________________________________________
lstm_3 (LSTM)                (210, 512)                2099200   
_________________________________________________________________
dense_2 (Dense)              (210, 64)                 32832     
_________________________________________________________________
dropout_1 (Dropout)          (210, 64)                 0         
_________________________________________________________________
dense_3 (Dense)              (210, 3)                  195       
Total params: 3,188,995
Trainable params: 3,188,995
Non-trainable params: 0
_________________________________________________________________


In [35]:
# compile model
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [75]:
X = all_melody_np
X = np.asarray(all_melody_np).astype(np.float32)
Y = np.array(all_harmony_np).reshape(210, -1, 3)
for note in X:
    print(type(note))
print(Y.shape)

ValueError: cannot reshape array of size 210 into shape (210,newaxis,3)

In [68]:
X = tf.convert_to_tensor(X, dtype=tf.float32)
Y = tf.convert_to_tensor(Y)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [65]:
print(all_melody_np.shape)
print(all_harmony_np.shape)
model = Sequential()
# 2 LSTM layers
model.add(LSTM(512, input_shape=(50, 3), return_sequences=True))
# model.add(LSTM(128, batch_input_shape=(None, None, 3), return_sequences=True))
model.add(LSTM(512))

# Dense layers
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))

# Output Layer
model.add(Dense(3, activation="softmax"))
model.summary()
model.compile(optimizer='adam', loss='mse')
print(model.summary())
trained_model = model.fit(X, Y, epochs=200, validation_split=0.2)

(210, 50, 3)
(210,)
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 50, 512)           1056768   
_________________________________________________________________
lstm_17 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_22 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 3)                 195       
Total params: 3,188,995
Trainable params: 3,188,995
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_11"
_________________________________________________________________


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
# train model
trained_model = model.fit(melody_train, harmony_train, validation_data=(melody_validation, harmony_validation),
          epochs=50, batch_size=1) # 32 is the default. Might play around with some of the parameters
trained_model.summary()

In [49]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, Dropout
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input

from tensorflow.keras.layers import Bidirectional

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [62]:
nums = 25

X1 = list()
X2 = list()
X = list()
Y = list()

X1 = [(x+1)*2 for x in range(25)]
X2 = [(x+1)*3 for x in range(25)]
Y = [x1*x2 for x1,x2 in zip(X1,X2)]


X = np.column_stack((X1, X2))
X = np.array(X).reshape(25, 1, 2)
Y = np.array(Y)
print(X.shape)
for i in X:
    print(type(i))
print(Y.shape)
for i in Y:
    print(type(i))
model = Sequential()
model.add(LSTM(80, activation='relu', input_shape=(1, 2)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())
#model.fit(X, Y, epochs=2000, validation_split=0.2, batch_size=5)
#test_input = array([55,80])
#test_input = test_input.reshape((1, 1, 2))
#test_output = model.predict(test_input, verbose=0)
#print(test_output)

(25, 1, 2)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(25,)
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'n